# Метод замены ядра на вырожденное

а) $\alpha = 0.5\\ \beta = 0.4$

$K(x,y)= cos((x-\alpha)(y-\beta))\qquad f(x)=e^{x/2}$

$K_2(x, y) = 1 - 0.5(x - \alpha)^2 (y - \beta)^2$

$\alpha_1(x)=1 \qquad \beta_1(y)=1\\
\alpha_2(x)=-0.5(x-\alpha)^2 \qquad \beta_2(y)=(y-\beta)^2$

$A_{ij}=\int_0^1\alpha_j(y)\beta_i(y)dy$

$A_{11}=\int_0^1dy=1$

$A_{12}=\int_0^1-0.5(y-0.5)^2dy=-\frac 1 6 + \frac 1 4 - \frac 1 8 = -\frac 1 {24}$

$A_{21}=\int_0^1(y-0.4)^2dy=\frac 1 3 - \frac 3 5 + \frac 4 {25} =  \frac 7 {75}$

$A_{22}=\int_0^1-0.5(y - 0.5)^2 (y - 0.4)^2dy=0.17 + \frac 9 {40} - \frac {121} {600} = \frac {29} {150}$

$A = \begin{vmatrix} 
        2 & -\frac 1 {24}\\
        \frac 7 {75} & 1 + \frac {29} {150} 
     \end{vmatrix}$

$\Delta = ||A|| = \frac {4303} {1800}$

$\Delta(x,y)=M_{11} - M_{12}(y-0.4)^2 + M_{21} \cdot 0.5(x-0.5)^2 - M_{22} \cdot 0.5(x-0.5)^2(y-0.4)^2$

$u(x)=f(x) -  \int_0^1 \frac {\Delta(x,y)} \Delta f(y) dy$

In [23]:
from scipy.integrate import quad
def integral_0_1(F):
    return quad(F, 0, 1)[0]

In [24]:
import numpy as np
import numpy.linalg as linalg

In [25]:
f = lambda x: np.exp(x / 2)
alpha = 0.5
beta = 0.4
a = [lambda x: 1, lambda x: -0.5*(x - alpha) ** 2]
b = [lambda y: 1, lambda y: (y - beta)**2]

In [26]:
def get_A(a, b):
    A_ij = lambda i,j: integral_0_1(lambda y:a[j](y) * b[i](y))
    n = len(a)
    A = np.array([[1 * (i == j) + A_ij(i,j) for j in range(n)] for i in range(n)])
    return A

In [27]:
def get_algebraic_addition(A, i, j):
    A = A.tolist()
    minor = [row[:j] + row[j+1:] for row in (A[:i] + A[i+1:])]
    return ((-1)**(i+j)) * np.array(minor)

In [28]:
def get_delta_x_y(A, a, b):
    n = len(a)
    return lambda x, y: np.array([[
                            get_algebraic_addition(A, i, j) * a[i](x) * b[j](y) 
                                for j in range(n)] 
                                  for i in range(n)]).sum()

In [29]:
def sub_method(a, b, f):
    A = get_A(a, b)
    delta = get_delta_x_y(A, a, b)
    u = lambda x: f(x) - integral_0_1(lambda y: delta(x, y) / linalg.norm(A) * f(y))
    return u

In [30]:
u = sub_method(a, b, f)

In [31]:
[u(i) for i in np.arange(0, 1.05, 0.05)]

[0.4468831708866653,
 0.46880301506833044,
 0.4917211120869268,
 0.5156536852722997,
 0.540617368649846,
 0.5666292173373194,
 0.5937067182048354,
 0.6218678008047342,
 0.6511308485781346,
 0.6815147103451823,
 0.713038712086177,
 0.7457226690209394,
 0.779586897993968,
 0.8146522301731276,
 0.8509400240698097,
 0.8884721788886943,
 0.9272711482154686,
 0.9673599540510469,
 1.0087622012010717,
 1.0515020920296845,
 1.0956044415867936]

In [32]:
def upper_integral_bound(f):
    F = lambda x: integral_0_1(lambda y: f(x, y))
    return max([F(i) for i in np.arange(0, 1.05, 0.05)])

In [33]:
K = lambda x, y: np.cos((x - alpha)*(y - beta))
K2 = lambda x, y: 1 - 0.5 * (x - alpha) ** 2 * (y - beta) ** 2

In [34]:
h = upper_integral_bound(lambda x, y: np.abs(K(x, y) - K2(x, y)))

In [35]:
def get_G(a, b):
    A = get_A(a, b)
    delta = get_delta_x_y(A, a, b)
    return lambda x, y: delta(x, y) / linalg.norm(A)

In [36]:
B = upper_integral_bound(get_G(a, b))

In [37]:
1 - h * (1 + B) > 0

True

In [38]:
C = upper_integral_bound(lambda x, y: np.abs(K(x, y)))

In [39]:
C < 1

False

In [40]:
def get_error(h, B, f):
    N = max([abs(f(i)) for i in np.arange(0, 1.05, 0.05)])
    return N * h * (1 + B) ** 2 / (1 - h * (1 + B))

In [41]:
get_error(h, B, f)

0.0001564913728597275